In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from Analyzer import Analyzer
import configparser
from glob import glob
from decimal import getcontext, Decimal
import json

class DecimalEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return str(obj)
        return json.JSONEncoder.default(self, obj)
    
if __name__ == '__main__':
    
    config = configparser.ConfigParser()
    config.read('config.txt')
    snapshots_files = sorted(glob(config["snapshot-export"]["path"]+"*.csv"))
    key = config["analyzer"]["key"]
    min_count = config["analyzer"].getint("min_count")
    version = config["report-export"]["report_version"]

    print("Start aggregating and analyzing...")
    pageAnalyzer = Analyzer(key, snapshots_files, config["report-export"]["path"])
    panel_snapshot = pageAnalyzer.load_accumulated_snapshot(filter_columns=["lead-Good", "lead-Bad"], min_count=min_count)
    pageAnalyzer.save_accumulated_snapshot() # store the cumulative result
    panel_report, bayesian_metrics, labelProportion = Analyzer.calc_metrics(panel_snapshot, key_column=key)
    
    # obtain export required metrics
    # getcontext().prec = 30
    # Pandas default precision
    
    labelProportion_Decimal = {k: Decimal(v) for k, v in labelProportion.items()}
    
    # pathMetrics = pd.concat([np.log(bayesian_metrics), panel_report["traffic"]], axis=1) # log version 
    pathMetrics = pd.concat([bayesian_metrics, panel_report["traffic"]], axis=1)
    columnSchema = list(pathMetrics.columns)

    pathMetrics_Decimal = pathMetrics.applymap(Decimal).T.to_dict("list")
    
    json_export = {
        "labelProportion": labelProportion_Decimal,
        "columnSchema": columnSchema,
        "pathMetrics": pathMetrics_Decimal
    }


    ################################
    # page_score json
    ################################
    with open("page_scores.json", 'wt', encoding='UTF-8') as f:
        json.dump(json_export, f, indent=4, cls=DecimalEncoder)
        
    print("Finished data analyzing and store the json data")
    
    ################################
    # valid url list
    ################################ 
    le = LabelEncoder()
    encode_urls = le.fit_transform(bayesian_metrics.index)
    np.save(f'./report/encode_urls_{version}.npy', le.classes_)
    print("largest code:", max(encode_urls))
    
    
    ################################
    # mcvisid report list
    ################################
    print("Processing probability report")
    preload = False
    unique_only = True
    if preload:
        request_list = pd.read_csv(f"./report/mcvisid_request_list_{version}.csv")
        request_list["page_code"] = request_list["page_code"].apply(eval)
    else: 
        test_sample_size = None
        files = sorted(glob("./data/aemRaw_keyColumns_2022*p1v1.csv.gz"))
        dfs = pd.DataFrame()
        for file in files:
            print("loading: ", file)
            df = pd.read_csv(file, compression="gzip", usecols=['mcvisid', 'clean_PageURL'], nrows=test_sample_size)
            dfs = pd.concat([dfs, df], axis=0)
        print("Finish loading")

        print("Aggregating user request list")    
        filter_dfs = dfs[dfs["clean_PageURL"].isin(le.classes_)]
        filter_dfs.loc[:, "page_code"] = le.transform(filter_dfs["clean_PageURL"].tolist())
        request_list = filter_dfs[["mcvisid", "page_code"]].groupby("mcvisid").apply(lambda x: x["page_code"].tolist()).rename("page_code").reset_index()
        request_list["unique_page_code"] = request_list["page_code"].apply(lambda x: list(set(x)))
        request_list["valid_request_count"] = request_list["unique_page_code"].apply(len)
        request_list.to_csv(f"./report/mcvisid_request_list_{version}.csv")
    print("Loaded request list report")
        
        
    if unique_only:
        request_list["page_code"] = request_list["unique_page_code"]
        request_list.drop(columns=["unique_page_code"],inplace=True)
    
    print("Processing probability")
    
    request_list_prob = Analyzer.mcvisid_probs(request_list, le, panel_report, bayesian_metrics, labelProportion)
    request_list_prob.to_csv(f"./report/mcvisid_request_list_probs-server_{version}.csv")
    # request_list_prob.to_excel("./report/mcvisid_request_list_probs.xlsx")
    target_list = request_list_prob[(request_list_prob["valid_request_count"]>5) & (request_list_prob["valid_request_count"]<500)]
    target_list.to_excel(f"./report/mcvisid_request_list_probs_target_{version}.xlsx")
    
    ################################
    # validation - merge mcvisid label to compare
    ################################
    


Start aggregating and analyzing...
loading ./snapshot/snapshot_20220401-20220415_p1v1.csv
loading ./snapshot/snapshot_20220415-20220430_p1v1.csv
loading ./snapshot/snapshot_20220501-20220515_p1v1.csv
loading ./snapshot/snapshot_20220515-20220531_p1v1.csv
loading ./snapshot/snapshot_20220601-20220615_p1v1.csv
loading ./snapshot/snapshot_20220615-20220630_p1v1.csv
loading ./snapshot/snapshot_20220701-20220715_p1v1.csv
loading ./snapshot/snapshot_20220715-20220731_p1v1.csv
loading ./snapshot/snapshot_20220801-20220815_p1v1.csv
loading ./snapshot/snapshot_20220815-20220831_p1v1.csv
loading ./snapshot/snapshot_20220901-20220915_p1v1.csv
loading ./snapshot/snapshot_20220915-20220930_p1v1.csv
loading ./snapshot/snapshot_20221001-20221015_p1v1.csv
loading ./snapshot/snapshot_20221015-20221031_p1v1.csv
loading ./snapshot/snapshot_20221101-20221115_p1v1.csv
loading ./snapshot/snapshot_20221115-20221130_p1v1.csv
loading ./snapshot/snapshot_20221201-20221215_p1v1.csv
loading ./snapshot/snapshot_20

/Users/wy/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


Loaded request list report
Processing probability


In [5]:
target_list = request_list_prob[(request_list_prob["valid_request_count"]>5) & (request_list_prob["valid_request_count"]<500)]

## sample checking

In [40]:
pd.set_option('precision', 100)

### calc code

In [24]:
request_list_filter = request_list[(request_list["valid_request_count"]>5) & (request_list["valid_request_count"]<100)]

In [47]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
conditional_parts = bayesian_metrics.multiply(panel_report["traffic"], axis="rows").div(labelProportion, axis=1) 
conditional_parts = conditional_parts[conditional_parts.index.isin(le.classes_)]
conditional_parts.index = le.transform(conditional_parts.index)

## obtain TF Matrix
vec = CountVectorizer(token_pattern="[\d]+", vocabulary=[str(i) for i in conditional_parts.index])
dtm = vec.fit_transform(request_list_filter["page_code"].astype(str))

## matrix calculation of Naive Bayesian
ratio1 = dtm * np.log(conditional_parts + 1e-321)
const = np.log(np.array([labelProportion[col] for col in conditional_parts.columns]) + 1e-321);
nominators = np.exp(ratio1 + const)
nominators_df = pd.DataFrame(nominators, index=request_list_filter["mcvisid"], columns=conditional_parts.columns)

## aggregate by aspect
probs = pd.DataFrame()
aspects = ["lead", "role", "industry"]
category_group = [[col for col in conditional_parts.columns if aspect in col ] for aspect in aspects]
for group in category_group:
    probs[group] = nominators_df[group].div(nominators_df[group].sum(axis=1) + 1e-321, axis=0)


## wrap back to DF
mcvisid_probs = pd.DataFrame(probs, index=request_list_filter["mcvisid"], columns=conditional_parts.columns)        

result = pd.concat([request_list_filter.set_index("mcvisid"), mcvisid_probs], axis=1)


In [27]:
target_list = result
labels = pd.read_csv("./data/updated_labels_20230131.csv.gz", compression="gzip")
df = target_list.merge(labels, on="mcvisid")

In [69]:
dff = df.dropna(subset=["lead-Good"]).sort_values("lead-Good")
# dff[(dff["label_lead"] == dff["lead-Good"]) & (dff["valid_request_count"]<10) & (dff["lead-Good"]>0.2) & (dff["lead-Good"]<0.9)]
aspect = ["lead-Bad", "lead-Good"]
samples = dff[(dff["label_lead"] == (dff["lead-Good"]>0.5)) & (dff["valid_request_count"]<10) & (dff["lead-Good"]>0.2) & (dff["lead-Good"]<0.9)].sort_values("lead-Good")
samples = samples[['mcvisid', 'page_code','lead-Good', 'lead-Bad',]]

In [71]:
samples.tail(5).style

,mcvisid,page_code,lead-Good,lead-Bad
56093,62189114061023640704252020270838166935,"[3080, 4267, 813, 240, 881, 243, 824, 4092]",0.8853535731941195585648074484197422862052917480468750000000000000000000000000000000000000000000000000,0.1146464268058805385797072062814550008624792098999023437500000000000000000000000000000000000000000000
18466,20528088904142157941257503168888003119,"[4320, 4289, 243, 4307, 4346, 4347]",0.8856363410118667234272038513154257088899612426757812500000000000000000000000000000000000000000000000,0.1143636589881332488172205330556607805192470550537109375000000000000000000000000000000000000000000000
30601,33901220597667739294452190890532306865,"[4097, 3080, 3099, 1068, 1069, 3536, 4085, 950, 1147]",0.8927875903717792382963125419337302446365356445312500000000000000000000000000000000000000000000000000,0.1072124096282207894592630736951832659542560577392578125000000000000000000000000000000000000000000000
58430,64815100308319927083853690183259639484,"[4002, 2980, 4004, 3181, 557, 243, 3988, 4085, 3639]",0.8935970094506886507446097311913035809993743896484375000000000000000000000000000000000000000000000000,0.1064029905493112937442390375508693978190422058105468750000000000000000000000000000000000000000000000
48682,53999771292848229630751307185745266414,"[4000, 3972, 3655, 4267, 814, 336, 881, 3988, 4085]",0.8976239128500264063958979932067450135946273803710937500000000000000000000000000000000000000000000000,0.1023760871499735242151629677209712099283933639526367187500000000000000000000000000000000000000000000


In [100]:
# good example
visited_codes = [3080, 4267, 813, 240, 881, 243, 824, 4092]
target_user = samples[samples["page_code"].map(set(visited_codes).issubset)]
print(target_user["mcvisid"].iloc[0], "visited pages: " , pathMetrics.iloc[visited_codes].index.tolist())

62189114061023640704252020270838166935 visited pages:  ['/products/hardware/allen-bradley.html', '/support/product/product-downloads.html', '/en-us.html', '/company/events.html', '/lang-selection.html', '/company/events/in-person-events/automation-fair.html', '/index.html', '/support.html']


In [90]:
pageKYields = pathMetrics.iloc[visited_codes][["lead-Good","lead-Bad", "traffic"]]
pageKYields.style

,lead-Good,lead-Bad,traffic
clean_PageURL,,,
/products/hardware/allen-bradley.html,0.3376659951040921758824708831525640562176704406738281250000000000000000000000000000000000000000000000,0.6623340048959077686063778855896089226007461547851562500000000000000000000000000000000000000000000000,0.0541177422276307509840798104505665833130478858947753906250000000000000000000000000000000000000000000
/support/product/product-downloads.html,0.3531431590068673798121778872882714495062828063964843750000000000000000000000000000000000000000000000,0.6468568409931325646766708814539015293121337890625000000000000000000000000000000000000000000000000000,0.0088771808268369410688247711505027837119996547698974609375000000000000000000000000000000000000000000
/en-us.html,0.3400936351247378919282482456765137612819671630859375000000000000000000000000000000000000000000000000,0.6599063648752621080717517543234862387180328369140625000000000000000000000000000000000000000000000000,0.1165669273429948277209788898289843928068876266479492187500000000000000000000000000000000000000000000
/company/events.html,0.4139344262295082121916323103505419567227363586425781250000000000000000000000000000000000000000000000,0.5860655737704918433195189209072850644588470458984375000000000000000000000000000000000000000000000000,0.0008581743747021449222889266650327044771984219551086425781250000000000000000000000000000000000000000
/lang-selection.html,0.3256191037735848836653929083695402368903160095214843750000000000000000000000000000000000000000000000,0.6743808962264150608234558603726327419281005859375000000000000000000000000000000000000000000000000000,0.0039766768838656769979866822950498317368328571319580078125000000000000000000000000000000000000000000
/company/events/in-person-events/automation-fair.html,0.3887577585453274031657144860218977555632591247558593750000000000000000000000000000000000000000000000,0.6112422414546726523454367452359292656183242797851562500000000000000000000000000000000000000000000000,0.0067053665793837674558020012227643746882677078247070312500000000000000000000000000000000000000000000
/index.html,0.8461538461538461453059767336526419967412948608398437500000000000000000000000000000000000000000000000,0.1538461538461538546940232663473580032587051391601562500000000000000000000000000000000000000000000000,0.0000076204008682567510351213088681809892932506045326590538024902343750000000000000000000000000000000
/support.html,0.3814886254042094693161857321683783084154129028320312500000000000000000000000000000000000000000000000,0.6185113745957905306838142678316216915845870971679687500000000000000000000000000000000000000000000000,0.0206653547853479606366544629736381466500461101531982421875000000000000000000000000000000000000000000


In [103]:
np.log(conditional_parts.iloc[visited_codes].iloc[:,0:2] + 1e-321).cumsum().style
# conditional_parts.iloc[visited_codes].iloc[:,0:2].cumprod().style

,lead-Good,lead-Bad
3080,-2.9562246134805838515546838607406243681907653808593750000000000000000000000000000000000000000000000000,-2.8957675448868425149839822552166879177093505859375000000000000000000000000000000000000000000000000000
4267,-7.6753109817883302667951284092850983142852783203125000000000000000000000000000000000000000000000000000,-7.6228581058225515221238310914486646652221679687500000000000000000000000000000000000000000000000000000
813,-9.8570683377489238807811489095911383628845214843750000000000000000000000000000000000000000000000000000,-9.7549941578276762044197312206961214542388916015625000000000000000000000000000000000000000000000000000
240,-16.7537526582091906846017081988975405693054199218750000000000000000000000000000000000000000000000000000,-16.9172100373474805223850125912576913833618164062500000000000000000000000000000000000000000000000000000
881,-22.3570217613186557059634651523083448410034179687500000000000000000000000000000000000000000000000000000,-22.4056680386456257281224679900333285331726074218750000000000000000000000000000000000000000000000000000
243,-27.2606010773375402322926674969494342803955078125000000000000000000000000000000000000000000000000000000,-27.4699661003180217733188328566029667854309082031250000000000000000000000000000000000000000000000000000
824,-38.1662701110687478944782924372702836990356445312500000000000000000000000000000000000000000000000000000,-40.6936388987658403948444174602627754211425781250000000000000000000000000000000000000000000000000000000
4092,-41.9631743847747813447313092183321714401245117187500000000000000000000000000000000000000000000000000000,-44.6205642974967418012965936213731765747070312500000000000000000000000000000000000000000000000000000000


In [99]:
a = nominators_df.loc[target_mcvisid].iloc[0:2]
a/a.sum()

lead-Good    0.88535357319411955856480744841974228620529174...
lead-Bad     0.11464642680588053857970720628145500086247920...
Name: 62189114061023640704252020270838166935, dtype: float64

In [ ]:
session variable
1. quite
2. not share cross


In [101]:
target_user

,mcvisid,page_code,lead-Good,lead-Bad
56093,62189114061023640704252020270838166935,"[3080, 4267, 813, 240, 881, 243, 824, 4092]",0.88535357319411955856480744841974228620529174...,0.11464642680588053857970720628145500086247920...


## classification report

In [3]:
from sklearn.metrics import classification_report
def evaluate_report(df, aspect, label_col, label_map, ignore_unknown=True):
    reverse_map = {v:k for k,v in label_map.items()}
    if ignore_unknown:
        data = df[df[label_col]!=-1]
    probs = data[aspect]
    pred = probs.columns[np.argmax(probs.values, axis=1)].tolist() # mapping into columns string
    label = data[label_col].map(reverse_map) # mapping into string 
    print(classification_report(label, pred, zero_division=1))
    return data


In [7]:
aspect = ["lead-Bad", "lead-Good"]
label_map = {"lead-Bad": 0, "lead-Good": 1}
label_col = "label_lead"
data = evaluate_report(df, aspect, label_col, label_map)

              precision    recall  f1-score   support

    lead-Bad       0.95      0.77      0.85     70611
   lead-Good       0.11      0.43      0.18      4727

    accuracy                           0.75     75338
   macro avg       0.53      0.60      0.52     75338
weighted avg       0.90      0.75      0.81     75338



In [8]:
# aspect = ["role-Other", "role-Engineer", "role-Manager", "role-Csuite"]
# label_col = "label_jobLevel"

# Standardized_JobLevel_Map = {
#         "role-Csuite": 4,
#         "role-Manager": 3,
#         "role-Engineer": 2,
#         "role-Marketing": -1,
#         "role-Unknown": 0,
#         "role-Other": -1,
#         }

# data = evaluate_report(df, aspect, label_col, Standardized_JobLevel_Map)

               precision    recall  f1-score   support

  role-Csuite       0.34      0.11      0.17      2308
role-Engineer       0.52      0.09      0.16     13413
 role-Manager       0.53      0.49      0.51     16006
   role-Other       0.00      1.00      0.00         0
 role-Unknown       1.00      0.00      0.00     16871

     accuracy                           0.19     48598
    macro avg       0.48      0.34      0.17     48598
 weighted avg       0.68      0.19      0.22     48598



In [105]:
le.transform(filter_dfs["clean_PageURL"])

array([ 813,  813,  813, ..., 3346, 3346,  810])

In [118]:
filter_dfs["clean_PageURL"].map(le_dict)

8           813
9           813
10          813
11          813
12          813
           ... 
1906079       1
1906080       1
1906083    3346
1906084    3346
1906085     810
Name: clean_PageURL, Length: 29114466, dtype: int64